In [2]:
import numpy

In [12]:
# Your existing setup
import can
import struct
import time
from datetime import datetime

def setup_can_bus(channel='can0', bitrate=500000):
    """Your existing setup function - no changes needed"""
    try:
        bus = can.interface.Bus(channel=channel, bustype='socketcan', bitrate=bitrate)
        print(f"Connected to CAN bus: {channel}")
        return bus
    except Exception as e:
        print(f"Error connecting to CAN bus: {e}")
        return None
setup_can_bus()


Connected to CAN bus: can0


/tmp/ipykernel_14003/3530546483.py:10: DeprecationWarning: The 'bustype' argument is deprecated since python-can v4.2.0, and scheduled for removal in python-can v5.0.0. Use 'interface' instead.
  bus = can.interface.Bus(channel=channel, bustype='socketcan', bitrate=bitrate)


/tmp/ipykernel_14003/834069698.py:215: DeprecationWarning: The 'bustype' argument is deprecated since python-can v4.2.0, and scheduled for removal in python-can v5.0.0. Use 'interface' instead.
  self.bus = can.interface.Bus(
Could not access SocketCAN device vcan0 ([Errno 19] No such device)



    To set up virtual CAN interface (run in terminal):
    
    sudo modprobe vcan
    sudo ip link add dev vcan0 type vcan
    sudo ip link set up vcan0
    
    To monitor CAN traffic:
    candump vcan0
    
    To use real CAN hardware, change channel to 'can0' or appropriate interface.
    
Failed to connect to CAN bus: [Errno 19] No such device


In [ ]:
# CAN SIL Testing in Jupyter Lab
# Interactive testing notebook for Cascadia motor controller CAN messages

# Cell 1: Setup and Installation
"""
First, install required packages:
!pip install python-can numpy

For virtual CAN setup (run in terminal):
sudo modprobe vcan
sudo ip link add dev vcan0 type vcan
sudo ip link set up vcan0

Monitor CAN traffic in another terminal:
candump vcan0
"""



In [13]:
# Cell 2: Import and Initialize
import sys
import time
import threading
from IPython.display import display, clear_output
import ipywidgets as widgets
from can_sil_test import CANSILTester, TestMode, setup_virtual_can

# Create global tester instance
tester = None

print("CAN SIL Tester for Jupyter Lab")
print("=" * 40)



CAN SIL Tester for Jupyter Lab


In [14]:
# Cell 3: Connection Management
def connect_to_can(channel="can0"):
    """Connect to CAN bus"""
    global tester
    tester = CANSILTester(channel)
    
    if tester.connect():
        print(f"✅ Connected to {channel}")
        return True
    else:
        print(f"❌ Failed to connect to {channel}")
        return False

def disconnect_can():
    """Disconnect from CAN bus"""
    global tester
    if tester:
        tester.stop()
        tester.disconnect()
        print("🔌 Disconnected from CAN bus")

# Connect by default
connect_to_can()



Connected to CAN bus: can0
✅ Connected to can0


True

In [15]:
# Cell 4: Basic Testing Functions
def start_normal_test(duration=30):
    """Start normal operation test"""
    if not tester:
        print("❌ Not connected to CAN bus")
        return
    
    print(f"🚀 Starting normal operation test for {duration}s...")
    tester.set_mode(TestMode.NORMAL_OPERATION)
    tester.set_send_rate(10)  # 10 Hz
    tester.start()
    
    # Status updates
    for i in range(duration):
        time.sleep(1)
        if i % 5 == 0:  # Update every 5 seconds
            clear_output(wait=True)
            print(f"🔄 Running normal test... {duration-i}s remaining")
            print(f"Motor Speed: {tester.simulator.motor_speed} RPM")
            print(f"Torque: {tester.simulator.torque_command:.1f} Nm")
    
    tester.stop()
    print("✅ Normal test completed")

def run_fault_injection_test(duration=15):
    """Test fault injection"""
    if not tester:
        print("❌ Not connected to CAN bus")
        return
    
    print(f"⚠️  Starting fault injection test for {duration}s...")
    tester.clear_faults()
    
    # Inject some specific faults
    tester.inject_fault("run_lo", 0)  # Hardware gate fault
    tester.inject_fault("run_lo", 5)  # Current sensor low
    tester.set_mode(TestMode.FAULT_INJECTION)
    tester.start()
    
    time.sleep(duration)
    
    tester.stop()
    tester.clear_faults()
    print("✅ Fault injection test completed")

def run_motor_acceleration_test(duration=25):
    """Test motor acceleration profile"""
    if not tester:
        print("❌ Not connected to CAN bus")
        return
    
    print(f"🏎️  Starting motor acceleration test for {duration}s...")
    tester.set_mode(TestMode.MOTOR_ACCELERATION)
    tester.start()
    
    for i in range(duration):
        time.sleep(1)
        if i % 3 == 0:
            clear_output(wait=True)
            print(f"🔄 Motor acceleration... {duration-i}s remaining")
            print(f"Speed: {tester.simulator.motor_speed} RPM")
            print(f"Torque: {tester.simulator.torque_command:.1f} Nm")
    
    tester.stop()
    print("✅ Motor acceleration test completed")



In [16]:
# Cell 5: Interactive Control Panel
def create_control_panel():
    """Create interactive control panel"""
    
    # Connection controls
    channel_input = widgets.Text(value="vcan0", description="CAN Channel:")
    connect_btn = widgets.Button(description="Connect", button_style="success")
    disconnect_btn = widgets.Button(description="Disconnect", button_style="danger")
    
    # Test mode selection
    mode_dropdown = widgets.Dropdown(
        options=[
            ("Normal Operation", TestMode.NORMAL_OPERATION),
            ("Fault Injection", TestMode.FAULT_INJECTION),
            ("Stress Test", TestMode.STRESS_TEST),
            ("Temperature Sweep", TestMode.TEMPERATURE_SWEEP),
            ("Voltage Ramp", TestMode.VOLTAGE_RAMP),
            ("Motor Acceleration", TestMode.MOTOR_ACCELERATION),
            ("Shutdown Sequence", TestMode.SHUTDOWN_SEQUENCE),
        ],
        value=TestMode.NORMAL_OPERATION,
        description="Test Mode:"
    )
    
    # Send rate control
    rate_slider = widgets.IntSlider(
        value=10, min=1, max=50, step=1,
        description="Send Rate (Hz):"
    )
    
    # Control buttons
    start_btn = widgets.Button(description="Start", button_style="success")
    stop_btn = widgets.Button(description="Stop", button_style="warning")
    status_btn = widgets.Button(description="Status", button_style="info")
    
    # Fault injection controls
    fault_type_dropdown = widgets.Dropdown(
        options=["post_lo", "post_hi", "run_lo", "run_hi"],
        description="Fault Type:"
    )
    fault_bit_input = widgets.IntText(value=0, description="Fault Bit:")
    inject_fault_btn = widgets.Button(description="Inject Fault", button_style="warning")
    clear_faults_btn = widgets.Button(description="Clear Faults", button_style="info")
    
    # Output area
    output = widgets.Output()
    
    # Event handlers
    def on_connect_clicked(b):
        with output:
            clear_output()
            connect_to_can(channel_input.value)
    
    def on_disconnect_clicked(b):
        with output:
            clear_output()
            disconnect_can()
    
    def on_start_clicked(b):
        with output:
            clear_output()
            if tester:
                tester.set_mode(mode_dropdown.value)
                tester.set_send_rate(rate_slider.value)
                tester.start()
            else:
                print("❌ Not connected to CAN bus")
    
    def on_stop_clicked(b):
        with output:
            clear_output()
            if tester:
                tester.stop()
            else:
                print("❌ No active tester")
    
    def on_status_clicked(b):
        with output:
            clear_output()
            if tester:
                tester.status()
            else:
                print("❌ Not connected to CAN bus")
    
    def on_inject_fault_clicked(b):
        with output:
            clear_output()
            if tester:
                tester.inject_fault(fault_type_dropdown.value, fault_bit_input.value)
            else:
                print("❌ Not connected to CAN bus")
    
    def on_clear_faults_clicked(b):
        with output:
            clear_output()
            if tester:
                tester.clear_faults()
            else:
                print("❌ Not connected to CAN bus")
    
    # Bind events
    connect_btn.on_click(on_connect_clicked)
    disconnect_btn.on_click(on_disconnect_clicked)
    start_btn.on_click(on_start_clicked)
    stop_btn.on_click(on_stop_clicked)
    status_btn.on_click(on_status_clicked)
    inject_fault_btn.on_click(on_inject_fault_clicked)
    clear_faults_btn.on_click(on_clear_faults_clicked)
    
    # Layout
    connection_box = widgets.HBox([channel_input, connect_btn, disconnect_btn])
    control_box = widgets.HBox([start_btn, stop_btn, status_btn])
    fault_box = widgets.HBox([fault_type_dropdown, fault_bit_input, inject_fault_btn, clear_faults_btn])
    
    panel = widgets.VBox([
        widgets.HTML("<h3>CAN SIL Tester Control Panel</h3>"),
        connection_box,
        mode_dropdown,
        rate_slider,
        control_box,
        widgets.HTML("<h4>Fault Injection</h4>"),
        fault_box,
        output
    ])
    
    return panel

# Create and display control panel
control_panel = create_control_panel()
display(control_panel)



Error sending message 0xa6: Failed to transmit: No buffer space available [Error Code 105]
Error sending message 0xa5: Failed to transmit: No buffer space available [Error Code 105]
Error sending message 0xac: Failed to transmit: No buffer space available [Error Code 105]
Error sending message 0xaa: Failed to transmit: No buffer space available [Error Code 105]
Error sending message 0xa0: Failed to transmit: No buffer space available [Error Code 105]
Error sending message 0xa1: Failed to transmit: No buffer space available [Error Code 105]
Error sending message 0xa2: Failed to transmit: No buffer space available [Error Code 105]
Error sending message 0xa7: Failed to transmit: No buffer space available [Error Code 105]
Error sending message 0xa6: Failed to transmit: No buffer space available [Error Code 105]
Error sending message 0xa5: Failed to transmit: No buffer space available [Error Code 105]
Error sending message 0xac: Failed to transmit: No buffer space available [Error Code 105]

In [17]:
# Cell 6: Predefined Test Sequences
def run_comprehensive_test():
    """Run a comprehensive test sequence"""
    if not tester:
        print("❌ Not connected to CAN bus")
        return
    
    print("🧪 Starting comprehensive test sequence...")
    
    test_sequence = [
        (TestMode.NORMAL_OPERATION, 10, "Normal operation baseline"),
        (TestMode.MOTOR_ACCELERATION, 15, "Motor acceleration profile"),
        (TestMode.TEMPERATURE_SWEEP, 12, "Temperature sweep test"),
        (TestMode.VOLTAGE_RAMP, 10, "Voltage ramp test"),
        (TestMode.STRESS_TEST, 8, "Stress test with random variations"),
        (TestMode.SHUTDOWN_SEQUENCE, 20, "Vehicle shutdown sequence"),
    ]
    
    for mode, duration, description in test_sequence:
        print(f"\n📋 {description} ({duration}s)")
        tester.set_mode(mode)
        tester.start()
        
        for i in range(duration):
            time.sleep(1)
            if i % 3 == 0:
                print(f"  ⏱️  {duration-i}s remaining - Speed: {tester.simulator.motor_speed} RPM")
        
        tester.stop()
        time.sleep(2)  # Brief pause between tests
    
    print("\n✅ Comprehensive test sequence completed!")



In [18]:
# Cell 7: Real-time Monitoring
def start_realtime_monitor(duration=60):
    """Start real-time monitoring with live updates"""
    if not tester:
        print("❌ Not connected to CAN bus")
        return
    
    print(f"📊 Starting real-time monitor for {duration}s...")
    
    tester.set_mode(TestMode.NORMAL_OPERATION)
    tester.start()
    
    import matplotlib.pyplot as plt
    from IPython.display import clear_output
    import numpy as np
    
    # Data storage
    times = []
    speeds = []
    torques = []
    temps = []
    
    start_time = time.time()
    
    plt.ion()
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 8))
    
    try:
        for _ in range(duration):
            current_time = time.time() - start_time
            times.append(current_time)
            speeds.append(tester.simulator.motor_speed)
            torques.append(tester.simulator.torque_command)
            temps.append(tester.simulator.temperatures['module_a'])
            
            # Keep only last 60 data points
            if len(times) > 60:
                times = times[-60:]
                speeds = speeds[-60:]
                torques = torques[-60:]
                temps = temps[-60:]
            
            # Clear and plot
            for ax in [ax1, ax2, ax3]:
                ax.clear()
            
            ax1.plot(times, speeds, 'b-', linewidth=2)
            ax1.set_ylabel('Speed (RPM)')
            ax1.set_title('Real-time Motor Data')
            ax1.grid(True)
            
            ax2.plot(times, torques, 'r-', linewidth=2)
            ax2.set_ylabel('Torque (Nm)')
            ax2.grid(True)
            
            ax3.plot(times, temps, 'g-', linewidth=2)
            ax3.set_ylabel('Temperature (°C)')
            ax3.set_xlabel('Time (s)')
            ax3.grid(True)
            
            plt.tight_layout()
            plt.draw()
            plt.pause(0.1)
            
            time.sleep(0.9)  # ~1 second total loop time
            
    except KeyboardInterrupt:
        print("\n⏹️  Monitoring stopped by user")
    finally:
        tester.stop()
        plt.ioff()
        plt.show()
        print("✅ Real-time monitoring completed")



In [19]:
# Cell 8: Quick Test Functions
def quick_tests():
    """Quick test buttons for common scenarios"""
    
    def test_normal():
        start_normal_test(15)
    
    def test_faults():
        run_fault_injection_test(10)
    
    def test_accel():
        run_motor_acceleration_test(20)
    
    # Create buttons
    normal_btn = widgets.Button(description="Normal Test (15s)", button_style="success")
    fault_btn = widgets.Button(description="Fault Test (10s)", button_style="warning")
    accel_btn = widgets.Button(description="Acceleration Test (20s)", button_style="info")
    comprehensive_btn = widgets.Button(description="Comprehensive Test", button_style="primary")
    monitor_btn = widgets.Button(description="Real-time Monitor (60s)", button_style="secondary")
    
    output = widgets.Output()
    
    # Event handlers
    def on_normal_clicked(b):
        with output:
            clear_output()
            test_normal()
    
    def on_fault_clicked(b):
        with output:
            clear_output()
            test_faults()
    
    def on_accel_clicked(b):
        with output:
            clear_output()
            test_accel()
    
    def on_comprehensive_clicked(b):
        with output:
            clear_output()
            run_comprehensive_test()
    
    def on_monitor_clicked(b):
        with output:
            clear_output()
            start_realtime_monitor()
    
    normal_btn.on_click(on_normal_clicked)
    fault_btn.on_click(on_fault_clicked)
    accel_btn.on_click(on_accel_clicked)
    comprehensive_btn.on_click(on_comprehensive_clicked)
    monitor_btn.on_click(on_monitor_clicked)
    
    button_box = widgets.HBox([normal_btn, fault_btn, accel_btn])
    advanced_box = widgets.HBox([comprehensive_btn, monitor_btn])
    
    quick_panel = widgets.VBox([
        widgets.HTML("<h3>Quick Tests</h3>"),
        button_box,
        advanced_box,
        output
    ])
    
    return quick_panel

# Display quick test panel
quick_panel = quick_tests()
display(quick_panel)



TraitError: The 'button_style' trait of a Button instance expected any of ['primary', 'success', 'info', 'warning', 'danger', ''] (case-insensitive), not the str 'secondary'.

In [20]:
# Cell 9: Cleanup
def cleanup():
    """Cleanup function to call when done"""
    global tester
    if tester:
        tester.stop()
        tester.disconnect()
    print("🧹 Cleanup completed")

# Register cleanup for notebook shutdown
import atexit
atexit.register(cleanup)

print("\n🎯 CAN SIL Tester is ready!")
print("📝 Use the control panels above to run tests")
print("🔍 Monitor CAN traffic with: candump vcan0")
print("🛑 Run cleanup() when finished")


🎯 CAN SIL Tester is ready!
📝 Use the control panels above to run tests
🔍 Monitor CAN traffic with: candump vcan0
🛑 Run cleanup() when finished
Error sending message 0xa6: Failed to transmit: No buffer space available [Error Code 105]
Error sending message 0xa5: Failed to transmit: No buffer space available [Error Code 105]
Error sending message 0xac: Failed to transmit: No buffer space available [Error Code 105]
Error sending message 0xaa: Failed to transmit: No buffer space available [Error Code 105]
Error sending message 0xa0: Failed to transmit: No buffer space available [Error Code 105]
Error sending message 0xa1: Failed to transmit: No buffer space available [Error Code 105]
Error sending message 0xa2: Failed to transmit: No buffer space available [Error Code 105]
Error sending message 0xa7: Failed to transmit: No buffer space available [Error Code 105]
Error sending message 0xa6: Failed to transmit: No buffer space available [Error Code 105]
Error sending message 0xa5: Failed to